# Base

- DICT_long_province_alphanumerics `{'province alphanumeric': 'province key', ...}`
- DICT_long_district_alphanumerics `{'Province English': {'district alphanumeric': 'district key', ...}}`
- DICT_long_ward_alphanumerics `{'Province English': {'District English': {'long ward alphanumeric': 'ward key'}}}` 

- LIST_long_district_alphanumerics `['district alphanumeric', ...]`
- LIST_long_ward_alphanumerics `['long ward alphanumeric', ...]`

- LIST_province_keys_1 `['short province key', ...]`
- LIST_province_keys_2 `['short province key', ...]`
- LIST_province_keys_3 `['short province key', ...]`

- DICT_province_map `{'short province key': {'province': 'Hồ Chí Minh', 'long_province': 'Thành phố Hồ Chí Minh', ...}, ...}`
- DICT_district_map `{'Province English': {'district key': {'level english': {'district': 'Tân Bình', 'long_district': 'Quận Tân Bình', ...}}}}`
- DICT_ward_map `{'Province English':{'District English': {'ward key': {'level english': {'ward': 'Phú Trinh', 'long_ward': 'Phường Phú Trinh', ...}}}}}`

- LIST_duplicated_district_province_keys `['short province key', ...]`
- DICT_duplicated_district_keys `['short district key', ...]`

- LIST_duplicated_ward_district_keys `['short district key', ...]`
- DICT_duplicated_ward_keys `['short ward key', ...]`


- DICT_unique_long_district_alphanumerics `{'long_district_alphanumeric': 'province_key'}`
- DICT_not_unique_long_district_alphanumerics
- LIST_contains_province_key_long_district_alphanumerics
- DICT_unique_district_keys `{'district_key': 'province_key'}`
- DICT_not_unique_district_keys `{'district_key': {'province_key 1': ['ward key', ...], {'province_key 2': ['ward key', ...]}}`

- DICT_alias_province_keys `{'alias province key': 'province key', ...}`


In [1]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
import pickle
import sys
sys.path.append('../../vietadminunits')
from utils import to_alphanumeric, to_key, abbreviate_alphanumeric_prefix
from tqdm.auto import tqdm

In [2]:
df = pd.read_csv('../../data/output/vietnam_administrative_units.csv')

In [3]:
df.shape

(10547, 20)

In [4]:
df['province_key'] = df['province'].apply(to_key, args=(1,))
df['district_key'] = df['short_district'].apply(to_key, args=(2,))
df['ward_key'] = df['short_ward'].apply(to_key, args=(3,))

df['long_province_alphanumeric'] = df['long_province'].apply(to_alphanumeric)
df['long_district_alphanumeric'] = df['long_district'].apply(to_alphanumeric)
df['long_ward_alphanumeric'] = df['long_ward'].apply(to_alphanumeric)

df['long_province_alphanumeric_english'] = df['long_province_english'].apply(to_alphanumeric)
df['long_district_alphanumeric_english'] = df['long_district_english'].apply(to_alphanumeric)
df['long_ward_alphanumeric_english'] = df['long_ward_english'].apply(to_alphanumeric)

df['province_alphanumeric'] = df['province'].apply(to_alphanumeric)
df['short_district_alphanumeric'] = df['short_district'].apply(to_alphanumeric)
df['short_ward_alphanumeric'] = df['short_ward'].apply(to_alphanumeric)

df['district_level_english'].fillna('', inplace=True)
df['ward_level_english'].fillna('', inplace=True)

## Create DICT long alphanumerics

In [5]:
DICT_long_province_alphanumerics = {}

for long_province_alphanumeric in df['long_province_alphanumeric'].unique().tolist():
    province_key = df.loc[df['long_province_alphanumeric']==long_province_alphanumeric, 'province_key'].values[0]
    DICT_long_province_alphanumerics[long_province_alphanumeric] = province_key
    if 'thanhpho' in long_province_alphanumeric:
        DICT_long_province_alphanumerics[long_province_alphanumeric.replace('thanhpho', 'tp.')] = province_key
        
for long_province_alphanumeric_english in df['long_province_alphanumeric_english'].unique().tolist():
    province_key = df.loc[df['long_province_alphanumeric_english']==long_province_alphanumeric_english, 'province_key'].values[0]
    DICT_long_province_alphanumerics[long_province_alphanumeric_english] = province_key

In [6]:
DICT_long_province_alphanumerics

{'thanhphohanoi': 'hanoi',
 'tp.hanoi': 'hanoi',
 'tinhhagiang': 'hagiang',
 'tinhcaobang': 'caobang',
 'tinhbackan': 'backan',
 'tinhtuyenquang': 'tuyenquang',
 'tinhlaocai': 'laocai',
 'tinhdienbien': 'dienbien',
 'tinhlaichau': 'laichau',
 'tinhsonla': 'sonla',
 'tinhyenbai': 'yenbai',
 'tinhhoabinh': 'hoabinh',
 'tinhthainguyen': 'thainguyen',
 'tinhlangson': 'langson',
 'tinhquangninh': 'quangninh',
 'tinhbacgiang': 'bacgiang',
 'tinhphutho': 'phutho',
 'tinhvinhphuc': 'vinhphuc',
 'tinhbacninh': 'bacninh',
 'tinhhaiduong': 'haiduong',
 'thanhphohaiphong': 'haiphong',
 'tp.haiphong': 'haiphong',
 'tinhhungyen': 'hungyen',
 'tinhthaibinh': 'thaibinh',
 'tinhhanam': 'hanam',
 'tinhnamdinh': 'namdinh',
 'tinhninhbinh': 'ninhbinh',
 'tinhthanhhoa': 'thanhhoa',
 'tinhnghean': 'nghean',
 'tinhhatinh': 'hatinh',
 'tinhquangbinh': 'quangbinh',
 'tinhquangtri': 'quangtri',
 'tinhthuathienhue': 'thuathienhue',
 'thanhphodanang': 'danang',
 'tp.danang': 'danang',
 'tinhquangnam': 'quangnam',

In [7]:
DICT_long_district_alphanumerics = {}
for province_english in df['province_english'].unique().tolist():
    district_data = {}
    for long_district_alphanumeric in df[df['province_english']==province_english]['long_district_alphanumeric'].unique().tolist():
        district_key = df.loc[(df['province_english']==province_english) & (df['long_district_alphanumeric']==long_district_alphanumeric), 'district_key'].values[0]
        district_data[long_district_alphanumeric] = district_key
        
        # alias_district_alphanumeric = re.sub(r'^thanhpho', 'tp.', long_district_alphanumeric, flags=re.IGNORECASE)
        # alias_district_alphanumeric = re.sub(r'^quan', 'q.', alias_district_alphanumeric, flags=re.IGNORECASE)
        # alias_district_alphanumeric = re.sub(r'^huyen', 'h.', alias_district_alphanumeric, flags=re.IGNORECASE)
        # alias_district_alphanumeric = re.sub(r'^thixa', 'tx.', alias_district_alphanumeric, flags=re.IGNORECASE)
        
        alias_district_alphanumeric = abbreviate_alphanumeric_prefix(long_district_alphanumeric)
        
        district_data[alias_district_alphanumeric] = district_key
        
    for long_district_alphanumeric_english in df[df['province_english']==province_english]['long_district_alphanumeric_english'].unique().tolist():
        district_key = df.loc[(df['province_english']==province_english) & (df['long_district_alphanumeric_english']==long_district_alphanumeric_english), 'district_key'].values[0]
        district_data[long_district_alphanumeric_english] = district_key
        
    DICT_long_district_alphanumerics[province_english] = district_data

In [8]:
DICT_long_district_alphanumerics

{'Ha Noi': {'quanbadinh': 'badinh',
  'q.badinh': 'badinh',
  'quanhoankiem': 'hoankiem',
  'q.hoankiem': 'hoankiem',
  'quantayho': 'tayho',
  'q.tayho': 'tayho',
  'quanlongbien': 'longbien',
  'q.longbien': 'longbien',
  'quancaugiay': 'caugiay',
  'q.caugiay': 'caugiay',
  'quandongda': 'dongda',
  'q.dongda': 'dongda',
  'quanhaibatrung': 'haibatrung',
  'q.haibatrung': 'haibatrung',
  'quanhoangmai': 'hoangmai',
  'q.hoangmai': 'hoangmai',
  'quanthanhxuan': 'thanhxuan',
  'q.thanhxuan': 'thanhxuan',
  'huyensocson': 'socson',
  'h.socson': 'socson',
  'huyendonganh': 'donganh',
  'h.donganh': 'donganh',
  'huyengialam': 'gialam',
  'h.gialam': 'gialam',
  'quannamtuliem': 'namtuliem',
  'q.namtuliem': 'namtuliem',
  'huyenthanhtri': 'thanhtri',
  'h.thanhtri': 'thanhtri',
  'quanbactuliem': 'bactuliem',
  'q.bactuliem': 'bactuliem',
  'huyenmelinh': 'melinh',
  'h.melinh': 'melinh',
  'quanhadong': 'hadong',
  'q.hadong': 'hadong',
  'thixasontay': 'sontay',
  'tt.sontay': 'sont

In [9]:
DICT_long_ward_alphanumerics = {}
for province_english in df['province_english'].unique().tolist():
    district_data = {}
    for district_english in df[df['province_english']==province_english]['district_english'].unique().tolist():
        ward_data = {}
        for long_ward_alphanumeric in df[(df['province_english']==province_english) & (df['district_english']==district_english)]['long_ward_alphanumeric'].dropna().unique().tolist():
            ward_key = df.loc[(df['province_english']==province_english) & (df['district_english']==district_english) & (df['long_ward_alphanumeric']==long_ward_alphanumeric), 'ward_key'].values[0]
            ward_data[long_ward_alphanumeric] = ward_key
            
            # alias_ward_alphanumeric = re.sub(r'^xa', 'x.', long_ward_alphanumeric, flags=re.IGNORECASE)
            # alias_ward_alphanumeric = re.sub(r'^phuong', 'p.', alias_ward_alphanumeric, flags=re.IGNORECASE)
            # alias_ward_alphanumeric = re.sub(r'^thitran', 'tt.', alias_ward_alphanumeric, flags=re.IGNORECASE)
            # alias_ward_alphanumeric = re.sub(r'^huyen', 'h.', alias_ward_alphanumeric, flags=re.IGNORECASE)
            
            alias_ward_alphanumeric = abbreviate_alphanumeric_prefix(long_ward_alphanumeric)
            
            ward_data[alias_ward_alphanumeric] = ward_key
        
        for long_ward_alphanumeric_english in df[(df['province_english']==province_english) & (df['district_english']==district_english)]['long_ward_alphanumeric_english'].dropna().unique().tolist():
            ward_key = df.loc[(df['province_english']==province_english) & (df['district_english']==district_english) & (df['long_ward_alphanumeric_english']==long_ward_alphanumeric_english), 'ward_key'].values[0]
            ward_data[long_ward_alphanumeric_english] = ward_key
        
        district_data[district_english] = ward_data
    
    DICT_long_ward_alphanumerics[province_english] = district_data

In [10]:
DICT_long_ward_alphanumerics

{'Ha Noi': {'Ba Dinh': {'phuongphucxa': 'phucxa',
   'p.phucxa': 'phucxa',
   'phuongtrucbach': 'trucbach',
   'p.trucbach': 'trucbach',
   'phuongvinhphuc': 'vinhphuc',
   'p.vinhphuc': 'vinhphuc',
   'phuongcongvi': 'congvi',
   'p.congvi': 'congvi',
   'phuonglieugiai': 'lieugiai',
   'p.lieugiai': 'lieugiai',
   'phuongnguyentrungtruc': 'nguyentrungtruc',
   'p.nguyentrungtruc': 'nguyentrungtruc',
   'phuongquanthanh': 'quanthanh',
   'p.quanthanh': 'quanthanh',
   'phuongngocha': 'ngocha',
   'p.ngocha': 'ngocha',
   'phuongdienbien': 'dienbien',
   'p.dienbien': 'dienbien',
   'phuongdoican': 'doican',
   'p.doican': 'doican',
   'phuongngockhanh': 'ngockhanh',
   'p.ngockhanh': 'ngockhanh',
   'phuongkimma': 'kimma',
   'p.kimma': 'kimma',
   'phuonggiangvo': 'giangvo',
   'p.giangvo': 'giangvo',
   'phuongthanhcong': 'thanhcong',
   'p.thanhcong': 'thanhcong',
   'phucxaward': 'phucxa',
   'trucbachward': 'trucbach',
   'vinhphucward': 'vinhphuc',
   'congviward': 'congvi',
   

In [11]:
long_district_alphanumerics = df['long_district_alphanumeric'].unique().tolist()
long_ward_alphanumerics = df['long_ward_alphanumeric'].dropna().unique().tolist()

We use LIST_long_ward_alphanumerics to remove a part in address before searching a key. But ward level make some mistake.
- Eg: "Thị xã Sơn Tây" bị mất do "Xã Sơn Tây" -> Không parse được district.
- Eg: "Nghĩa Xá Lê Chân Hải Phòng" Bị mất "Xá Lê Chân H" do "Xã Lê Chánh" -> Không parse được province

In [12]:
len(long_ward_alphanumerics)

7366

In [13]:
for ward in ['long_ward_alphanumeric', 'short_ward_alphanumeric']:
    for district in ['long_district_alphanumeric', 'short_district_alphanumeric']:
        for province in ['long_province_alphanumeric', 'province_alphanumeric']:
            df[f"address_{ward}_{district}_{province}"] = df[ward] + df[district] + df[province]
            df[f"address_comma_{ward}_{district}_{province}"] = df[ward] + ',' + df[district] + ',' + df[province]
            df[f"address_{ward}_{province}_{district}"] = df[ward] + df[province] + df[district]
            df[f"address_comma_{ward}_{province}_{district}"] = df[ward] + ',' + df[province] + ',' + df[district]
               
for ward in ['long_ward_alphanumeric', 'short_ward_alphanumeric']:
    for district in ['long_district_alphanumeric', 'short_district_alphanumeric']:
        df[f"address_{ward}_add_tinh_{district}"] = df[ward] + 'tinh' + df[district]
        # xadongtinhvinhphuctamduong remove "tinhvinhphuc" con "xadongtamduong" sau do bi remove "xadongtam" -> Khong tim duoc tam duong
        df[f"address_{ward}_remove_tinh_{district}"] = df[ward].str.replace(r'tinh$', '', regex=True) + df[district]

In [14]:
address_cols = [col for col in df.columns if 'address' in col]

In [15]:
LIST_safe_long_ward_alphanumerics = []
for long_ward_alphanumeric in tqdm(long_ward_alphanumerics):
    tmp_df = df[df.long_ward_alphanumeric!=long_ward_alphanumeric][address_cols]
    addresses = str(tmp_df.values.tolist())
    if long_ward_alphanumeric not in addresses:
        LIST_safe_long_ward_alphanumerics.append(long_ward_alphanumeric)

  0%|          | 0/7366 [00:00<?, ?it/s]

In [16]:
len(LIST_safe_long_ward_alphanumerics)

6646

In [17]:
LIST_safe_long_ward_alphanumerics = LIST_safe_long_ward_alphanumerics + [abbreviate_alphanumeric_prefix(i) for i in LIST_safe_long_ward_alphanumerics]

In [18]:
LIST_safe_long_district_alphanumerics = []
for long_district_alphanumeric in tqdm(long_district_alphanumerics):
    tmp_df = df[df.long_district_alphanumeric!=long_district_alphanumeric][address_cols]
    addresses = str(tmp_df.values.tolist())
    if long_district_alphanumeric not in addresses:
        LIST_safe_long_district_alphanumerics.append(long_district_alphanumeric)

  0%|          | 0/685 [00:00<?, ?it/s]

In [19]:
len(LIST_safe_long_district_alphanumerics)

671

In [20]:
LIST_safe_long_district_alphanumerics = LIST_safe_long_district_alphanumerics + [abbreviate_alphanumeric_prefix(i) for i in LIST_safe_long_district_alphanumerics]

In [21]:
df.drop(columns=address_cols, inplace=True)

## Create short key & map

### Sort province

In [22]:
province_keys = df['province_key'].unique().tolist()
district_keys = df['district_key'].unique().tolist()
ward_keys = df['ward_key'].unique().tolist()

In [23]:
print('province_keys:', len(province_keys))
print('district_keys:', len(district_keys))
print('ward_keys:', len(ward_keys))

province_keys: 63
district_keys: 669
ward_keys: 6648


In [24]:
def find_province_key_match(text):
    for province_key in province_keys:
        if province_key in text:
            return province_key

In [25]:
containing_province_districts = df[df.district_key.str.contains('|'.join(province_keys))][['province', 'district', 'province_key','district_key']].drop_duplicates()
containing_province_wards = df[(~df.ward.isna()) & (df.ward_key.str.contains('|'.join(province_keys)))][['province', 'ward', 'province_key','ward_key']].drop_duplicates()

containing_province_districts['match_province_key'] = containing_province_districts['district_key'].apply(find_province_key_match)
containing_province_wards['match_province_key'] = containing_province_wards['ward_key'].apply(find_province_key_match)

containing_province_districts = containing_province_districts[containing_province_districts.province_key != containing_province_districts.match_province_key]
containing_province_wards = containing_province_wards[containing_province_wards.province_key != containing_province_wards.match_province_key]

In [26]:
containing_province_districts

,province,district,province_key,district_key,match_province_key
757,Hà Giang,Quang Bình,hagiang,quangbinh,quangbinh
1649,Sơn La,Phù Yên,sonla,phuyen,phuyen
3824,Hưng Yên,Văn Giang,hungyen,vangiang,angiang
5972,Quảng Bình,Quảng Ninh,quangbinh,quangninh,quangninh
9009,Long An,Thạnh Hóa,longan,thanhhoa,thanhhoa
10438,Bạc Liêu,Hòa Bình,baclieu,hoabinh,hoabinh


In [27]:
containing_province_wards

,province,ward,province_key,ward_key,match_province_key
2,Hà Nội,Vĩnh Phúc,hanoi,vinhphuc,vinhphuc
8,Hà Nội,Điện Biên,hanoi,dienbien,dienbien
142,Hà Nội,Xuân Giang,hanoi,xuangiang,angiang
316,Hà Nội,Yên Bài,hanoi,yenbai,yenbai
353,Hà Nội,Đồng Tháp,hanoi,dongthap,dongthap
...,...,...,...,...,...
10154,Cần Thơ,Thạnh Hòa,cantho,thanhhoa,thanhhoa
10236,Hậu Giang,Thạnh Hòa,haugiang,thanhhoa,thanhhoa
10363,Sóc Trăng,Khánh Hòa,soctrang,khanhhoa,khanhhoa
10438,Bạc Liêu,Hòa Bình,baclieu,hoabinh,hoabinh


In [28]:
# Should not use list(set()) because it will re-order
province_key_order = []

for row in containing_province_districts.itertuples():
    if row.province_key not in province_key_order:
        province_key_order.append(row.province_key)
    if row.match_province_key not in province_key_order:
        province_key_order.append(row.match_province_key)
        
for row in containing_province_wards.itertuples():
    if row.province_key not in province_key_order:
        province_key_order.append(row.province_key)
    if row.match_province_key not in province_key_order:
        province_key_order.append(row.match_province_key)

In [29]:
province_key_zero = [province_key for province_key in province_keys if province_key not in province_key_order]
province_key_zero = sorted(province_key_zero, key=lambda x: len(x), reverse=True)

In [30]:
province_key_order = province_key_zero + province_key_order

In [31]:
len(province_key_order)

63

In [32]:
province_key_order_map = {}
for index, province_key in enumerate(province_key_order):
    province_key_order_map[province_key] = index + 1

In [33]:
province_key_order_map

{'ninhthuan': 1,
 'laocai': 2,
 'bentre': 3,
 'hagiang': 4,
 'quangbinh': 5,
 'sonla': 6,
 'phuyen': 7,
 'hungyen': 8,
 'angiang': 9,
 'quangninh': 10,
 'longan': 11,
 'thanhhoa': 12,
 'baclieu': 13,
 'hoabinh': 14,
 'hanoi': 15,
 'vinhphuc': 16,
 'dienbien': 17,
 'yenbai': 18,
 'dongthap': 19,
 'caobang': 20,
 'binhduong': 21,
 'backan': 22,
 'phutho': 23,
 'tuyenquang': 24,
 'thaibinh': 25,
 'laichau': 26,
 'khanhhoa': 27,
 'binhthuan': 28,
 'thainguyen': 29,
 'langson': 30,
 'bacgiang': 31,
 'hanam': 32,
 'binhdinh': 33,
 'bacninh': 34,
 'haiphong': 35,
 'lamdong': 36,
 'vinhlong': 37,
 'tayninh': 38,
 'haugiang': 39,
 'namdinh': 40,
 'haiduong': 41,
 'ninhbinh': 42,
 'nghean': 43,
 'hatinh': 44,
 'kiengiang': 45,
 'quangtri': 46,
 'thuathienhue': 47,
 'danang': 48,
 'quangnam': 49,
 'travinh': 50,
 'quangngai': 51,
 'binhphuoc': 52,
 'kontum': 53,
 'daknong': 54,
 'gialai': 55,
 'daklak': 56,
 'dongnai': 57,
 'bariavungtau': 58,
 'hochiminh': 59,
 'tiengiang': 60,
 'cantho': 61,
 '

In [34]:
df['province_key_order'] = df['province_key'].map(province_key_order_map).fillna(0)
df.sort_values(by='province_key_order', inplace=True)
df.drop(columns=['province_key_order'], inplace=True)
province_keys = df['province_key'].unique().tolist() # Re-create after sorting

### Create key & map

In [35]:
LIST_province_keys_1 = []
LIST_province_keys_2 = []
LIST_province_keys_3 = []

for province_key in province_keys:
    # Unique province key
    if province_key not in str(district_keys) and province_key not in str(ward_keys):
        LIST_province_keys_1.append(province_key)
    # Same key with district
    elif province_key in str(district_keys):
        LIST_province_keys_2.append(province_key)
    # Same key with ward
    else:
        LIST_province_keys_3.append(province_key)

In [36]:
df_provinces = df[[col for col in df.columns if 'province' in col]].drop_duplicates()

In [37]:
df_provinces

,province,long_province,province_english,long_province_english,province_key,long_province_alphanumeric,long_province_alphanumeric_english,province_alphanumeric
7229,Ninh Thuận,Tỉnh Ninh Thuận,Ninh Thuan,Ninh Thuan Province,ninhthuan,tinhninhthuan,ninhthuanprovince,ninhthuan
1280,Lào Cai,Tỉnh Lào Cai,Lao Cai,Lao Cai Province,laocai,tinhlaocai,laocaiprovince,laocai
9435,Bến Tre,Tỉnh Bến Tre,Ben Tre,Ben Tre Province,bentre,tinhbentre,bentreprovince,bentre
645,Hà Giang,Tỉnh Hà Giang,Ha Giang,Ha Giang Province,hagiang,tinhhagiang,hagiangprovince,hagiang
5987,Quảng Bình,Tỉnh Quảng Bình,Quang Binh,Quang Binh Province,quangbinh,tinhquangbinh,quangbinhprovince,quangbinh
...,...,...,...,...,...,...,...,...
8857,Hồ Chí Minh,Thành phố Hồ Chí Minh,Ho Chi Minh,Ho Chi Minh City,hochiminh,thanhphohochiminh,hochiminhcity,hochiminh
9242,Tiền Giang,Tỉnh Tiền Giang,Tien Giang,Tien Giang Province,tiengiang,tinhtiengiang,tiengiangprovince,tiengiang
10160,Cần Thơ,Thành phố Cần Thơ,Can Tho,Can Tho City,cantho,thanhphocantho,canthocity,cantho
10304,Sóc Trăng,Tỉnh Sóc Trăng,Soc Trang,Soc Trang Province,soctrang,tinhsoctrang,soctrangprovince,soctrang


In [38]:
DICT_province_map = {}
for province_key in df_provinces.province_key.unique():
    df_province = df_provinces[df_provinces['province_key'] == province_key]
    province_record = df_province.to_dict(orient='records')[0]
    DICT_province_map[province_key] = province_record

In [39]:
DICT_province_map

{'ninhthuan': {'province': 'Ninh Thuận',
  'long_province': 'Tỉnh Ninh Thuận',
  'province_english': 'Ninh Thuan',
  'long_province_english': 'Ninh Thuan Province',
  'province_key': 'ninhthuan',
  'long_province_alphanumeric': 'tinhninhthuan',
  'long_province_alphanumeric_english': 'ninhthuanprovince',
  'province_alphanumeric': 'ninhthuan'},
 'laocai': {'province': 'Lào Cai',
  'long_province': 'Tỉnh Lào Cai',
  'province_english': 'Lao Cai',
  'long_province_english': 'Lao Cai Province',
  'province_key': 'laocai',
  'long_province_alphanumeric': 'tinhlaocai',
  'long_province_alphanumeric_english': 'laocaiprovince',
  'province_alphanumeric': 'laocai'},
 'bentre': {'province': 'Bến Tre',
  'long_province': 'Tỉnh Bến Tre',
  'province_english': 'Ben Tre',
  'long_province_english': 'Ben Tre Province',
  'province_key': 'bentre',
  'long_province_alphanumeric': 'tinhbentre',
  'long_province_alphanumeric_english': 'bentreprovince',
  'province_alphanumeric': 'bentre'},
 'hagiang': {

In [40]:
df_districts = df[['province_english'] + [col for col in df.columns if 'district' in col]].drop_duplicates()

In [41]:
df_districts

,province_english,district,long_district,short_district,district_english,long_district_english,short_district_english,district_level,district_level_english,district_key,long_district_alphanumeric,long_district_alphanumeric_english,short_district_alphanumeric
7229,Ninh Thuan,Thuận Bắc,Huyện Thuận Bắc,Thuận Bắc,Thuan Bac,Thuan Bac District,Thuan Bac,Huyện,District,thuanbac,huyenthuanbac,thuanbacdistrict,thuanbac
7211,Ninh Thuan,Ninh Hải,Huyện Ninh Hải,Ninh Hải,Ninh Hai,Ninh Hai District,Ninh Hai,Huyện,District,ninhhai,huyenninhhai,ninhhaidistrict,ninhhai
7208,Ninh Thuan,Ninh Sơn,Huyện Ninh Sơn,Ninh Sơn,Ninh Son,Ninh Son District,Ninh Son,Huyện,District,ninhson,huyenninhson,ninhsondistrict,ninhson
7200,Ninh Thuan,Bác Ái,Huyện Bác Ái,Bác Ái,Bac Ai,Bac Ai District,Bac Ai,Huyện,District,bacai,huyenbacai,bacaidistrict,bacai
7191,Ninh Thuan,Phan Rang - Tháp Chàm,Thành phố Phan Rang - Tháp Chàm,Phan Rang - Tháp Chàm,Phan Rang - Thap Cham,Phan Rang - Thap Cham City,Phan Rang - Thap Cham,Thành phố,City,phanrangthapcham,thanhphophanrangthapcham,phanrangthapchamcity,phanrangthapcham
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10530,Ca Mau,Năm Căn,Huyện Năm Căn,Năm Căn,Nam Can,Nam Can District,Nam Can,Huyện,District,namcan,huyennamcan,namcandistrict,namcan
10465,Ca Mau,U Minh,Huyện U Minh,U Minh,U Minh,U Minh District,U Minh,Huyện,District,uminh,huyenuminh,uminhdistrict,uminh
10494,Ca Mau,Trần Văn Thời,Huyện Trần Văn Thời,Trần Văn Thời,Tran Van Thoi,Tran Van Thoi District,Tran Van Thoi,Huyện,District,tranvanthoi,huyentranvanthoi,tranvanthoidistrict,tranvanthoi
10462,Ca Mau,Cà Mau,Thành phố Cà Mau,Cà Mau,Ca Mau,Ca Mau City,Ca Mau,Thành phố,City,camau,thanhphocamau,camaucity,camau


In [42]:
DICT_district_map = {}
for province_english in df_districts['province_english'].unique():
    district_data = {}
    for district_key in df_districts[df_districts['province_english'] == province_english]['district_key'].unique():
        df_district = df_districts[(df_districts['province_english'] == province_english) & (df_districts['district_key'] == district_key)].drop(columns=['province_english'])
        district_levels = df_district['district_level_english'].unique()
        level_data = {}
        for level in district_levels:
            district_record = df_district[df_district['district_level_english'] == level].to_dict(orient='records')[0]
            level_data[level] = district_record
            
        district_data[district_key] = level_data
        if re.search(r'^quan\d{1,2}', district_key):
            district_data[district_key.replace('quan', 'q.')] = level_data
            district_data[district_key.replace('quan', 'district')] = level_data
    DICT_district_map[province_english] = district_data

In [43]:
DICT_district_map

{'Ninh Thuan': {'thuanbac': {'District': {'district': 'Thuận Bắc',
    'long_district': 'Huyện Thuận Bắc',
    'short_district': 'Thuận Bắc',
    'district_english': 'Thuan Bac',
    'long_district_english': 'Thuan Bac District',
    'short_district_english': 'Thuan Bac',
    'district_level': 'Huyện',
    'district_level_english': 'District',
    'district_key': 'thuanbac',
    'long_district_alphanumeric': 'huyenthuanbac',
    'long_district_alphanumeric_english': 'thuanbacdistrict',
    'short_district_alphanumeric': 'thuanbac'}},
  'ninhhai': {'District': {'district': 'Ninh Hải',
    'long_district': 'Huyện Ninh Hải',
    'short_district': 'Ninh Hải',
    'district_english': 'Ninh Hai',
    'long_district_english': 'Ninh Hai District',
    'short_district_english': 'Ninh Hai',
    'district_level': 'Huyện',
    'district_level_english': 'District',
    'district_key': 'ninhhai',
    'long_district_alphanumeric': 'huyenninhhai',
    'long_district_alphanumeric_english': 'ninhhaidist

In [44]:
DICT_ward_map = {}
for province_english in df['province_english'].unique():
    district_data = {}
    for district_english in df[df['province_english'] == province_english]['district_english'].unique():
        ward_data = {}
        for ward_key in df[(df['province_english'] == province_english) & (df['district_english'] == district_english)]['ward_key'].dropna().unique():
            df_ward = df[(df['province_english'] == province_english) & (df['district_english'] == district_english) & (df['ward_key'] == ward_key)][[col for col in df.columns if 'ward' in col]]
            ward_levels = df_ward['ward_level_english'].unique()
            level_data = {}
            for level in ward_levels:
                ward_record = df_ward[df_ward['ward_level_english']==level].to_dict(orient='records')[0]
                level_data[level] = ward_record
                
            ward_data[ward_key] = level_data
            if re.search(r'^phuong\d{1,2}', ward_key):
                ward_data[ward_key.replace('phuong', 'p.')] = level_data
                ward_data[ward_key.replace('phuong', 'ward')] = level_data
            
        district_data[district_english] = ward_data
    DICT_ward_map[province_english] = district_data

In [45]:
DICT_ward_map['Ho Chi Minh']

{'District 8': {'phuong14': {'Ward': {'ward': 'Phường 14',
    'long_ward': 'Phường 14',
    'short_ward': 'Phường 14',
    'ward_english': 'Ward 14',
    'long_ward_english': 'Ward 14',
    'short_ward_english': 'Ward 14',
    'ward_level': 'Phường',
    'ward_level_english': 'Ward',
    'ward_key': 'phuong14',
    'long_ward_alphanumeric': 'phuong14',
    'long_ward_alphanumeric_english': 'ward14',
    'short_ward_alphanumeric': 'phuong14'}},
  'p.14': {'Ward': {'ward': 'Phường 14',
    'long_ward': 'Phường 14',
    'short_ward': 'Phường 14',
    'ward_english': 'Ward 14',
    'long_ward_english': 'Ward 14',
    'short_ward_english': 'Ward 14',
    'ward_level': 'Phường',
    'ward_level_english': 'Ward',
    'ward_key': 'phuong14',
    'long_ward_alphanumeric': 'phuong14',
    'long_ward_alphanumeric_english': 'ward14',
    'short_ward_alphanumeric': 'phuong14'}},
  'ward14': {'Ward': {'ward': 'Phường 14',
    'long_ward': 'Phường 14',
    'short_ward': 'Phường 14',
    'ward_englis

## Create duplicate district keys checking data

### District

In [47]:
district_count = df[['province', 'long_district', 'district_key']].drop_duplicates()[['province', 'district_key']].value_counts().reset_index()
duplicated_districts = district_count[district_count['count'] > 1]['district_key'].tolist()

In [59]:
df_duplicated_districts = df[df.district_key.isin(duplicated_districts)][['long_province', 'long_district' ,'province_key','district_key', 'province_english']].drop_duplicates()

In [60]:
df_duplicated_districts

,long_province,long_district,province_key,district_key,province_english
9748,Tỉnh Đồng Tháp,Huyện Cao Lãnh,dongthap,caolanh,Dong Thap
9680,Tỉnh Đồng Tháp,Thành phố Cao Lãnh,dongthap,caolanh,Dong Thap
9721,Tỉnh Đồng Tháp,Huyện Hồng Ngự,dongthap,hongngu,Dong Thap
9698,Tỉnh Đồng Tháp,Thành phố Hồng Ngự,dongthap,hongngu,Dong Thap
10267,Tỉnh Hậu Giang,Thị xã Long Mỹ,haugiang,longmy,Hau Giang
10264,Tỉnh Hậu Giang,Huyện Long Mỹ,haugiang,longmy,Hau Giang
5870,Tỉnh Hà Tĩnh,Thị xã Kỳ Anh,hatinh,kyanh,Ha Tinh
5842,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,hatinh,kyanh,Ha Tinh
9552,Tỉnh Trà Vinh,Huyện Duyên Hải,travinh,duyenhai,Tra Vinh
9559,Tỉnh Trà Vinh,Thị xã Duyên Hải,travinh,duyenhai,Tra Vinh


My idea: We will use their wards to decide their district level.

To make sure this idea is valid, I will check whether they have some wards with the same ward_key

In [50]:
for district_key in duplicated_districts:
    long_districts = df[df.district_key==district_key]['long_district'].unique().tolist()
    a = []
    b = []
    long_district_a = long_districts[0]
    long_district_b = long_districts[1]
    ward_key_a = df[df.long_district==long_district_a]['ward_key'].unique().tolist()
    ward_key_b = df[df.long_district==long_district_b]['ward_key'].unique().tolist()
    common = list(set(a) & set(b))
    if common:
        print(district_key, 'has duplicated ward_key')

Nice! We also have default option if there are no ward in the address, I use Google Trend to decide default district level.

In [51]:
# LIST_duplicated_district_province_keys = df_duplicated_districts['province_key'].unique().tolist()
# 
# # Use Google Trend and search to decide level
# DICT_duplicated_district_keys = {
#     'kyanh': {'default':'Town'},
#     'cailay': {'default':'Town'},
#     'duyenhai': {'default':'District'},
#     'caolanh': {'default':'City'},
#     'hongngu': {'default':'City'},
#     'longmy': {'default':'District'}
# }
# 
# for district_key in duplicated_districts:
#     df_temp = df[df.district_key == district_key]
#     level_data = {}
#     levels = df_temp.district_level_english.unique().tolist()
#     for district_level_english in levels:
#         tmp_ward_keys = df_temp[df_temp.district_level_english==district_level_english]['ward_key'].unique().tolist()
#         level_data[district_level_english] = tmp_ward_keys
#         DICT_duplicated_district_keys[district_key]['levels'] = level_data

In [71]:
default_option_for_duplicated_district_keys = {
    'kyanh': 'Town',
    'cailay': 'Town',
    'duyenhai': 'District',
    'caolanh': 'City',
    'hongngu': 'City',
    'longmy': 'District'
}

In [72]:
DICT_duplicated_district_keys = {}
for province_english in df_duplicated_districts.province_english.unique():
    tmp_df = df_duplicated_districts[df_duplicated_districts.province_english == province_english]
    tmp_duplicated_districts = tmp_df['district_key'].unique().tolist()
    district_data = {}
    for district_key in tmp_duplicated_districts:
        tmp_df_district = df[(df.district_key == district_key) & (df.province_english==province_english)]
        level_data = {}
        levels = tmp_df_district.district_level_english.unique().tolist()
        for district_level_english in levels:
            tmp_ward_keys = tmp_df_district[tmp_df_district.district_level_english==district_level_english]['ward_key'].unique().tolist()
            level_data[district_level_english] = tmp_ward_keys
            
        district_data[district_key] = {
            'default': default_option_for_duplicated_district_keys[district_key],
            'levels': level_data
        }
            
    DICT_duplicated_district_keys[province_english] = district_data

In [69]:
DICT_duplicated_district_keys['Ha Tinh']['kyanh']['default']

'Town'

In [70]:
DICT_duplicated_district_keys['Ha Tinh']['kyanh']['levels']

{'Town': ['kyha',
  'kyloi',
  'kyninh',
  'hungtri',
  'kytrinh',
  'kythinh',
  'kyphuong',
  'kyhoa',
  'kylong',
  'kylien',
  'kynam'],
 'District': ['kykhang',
  'kyvan',
  'kytrung',
  'kytho',
  'kytay',
  'kylac',
  'kyson',
  'lamhop',
  'kytan',
  'kythuong',
  'kyhai',
  'kythu',
  'kychau',
  'kydong',
  'kygiang',
  'kytien',
  'kyphu',
  'kybac',
  'kyxuan',
  'kyphong']}

### Ward

In [86]:
ward_count = df[['long_province', 'long_district', 'short_ward']].value_counts().reset_index()
df_duplicated_wards = ward_count[ward_count['count'] > 1].copy()
df_duplicated_wards = df[(df.long_province.isin(df_duplicated_wards.long_province)) & (df.long_district.isin(df_duplicated_wards.long_district)) & (df.short_ward.isin(df_duplicated_wards.short_ward))][['province', 'long_district', 'ward','ward_level_english', 'district_key', 'ward_key', 'province_english', 'district_english']]
df_duplicated_wards.sort_values(by=df_duplicated_wards.columns.tolist(), inplace=True)

In [87]:
df_duplicated_wards

,province,long_district,ward,ward_level_english,district_key,ward_key,province_english,district_english
10401,Bạc Liêu,Huyện Phước Long,Thị trấn Phước Long,Town,phuoclong,phuoclong,Bac Lieu,Phuoc Long
10404,Bạc Liêu,Huyện Phước Long,Xã Phước Long,Commune,phuoclong,phuoclong,Bac Lieu,Phuoc Long
10471,Cà Mau,Huyện Thới Bình,Thị trấn Thới Bình,Town,thoibinh,thoibinh,Ca Mau,Thoi Binh
10477,Cà Mau,Huyện Thới Bình,Xã Thới Bình,Commune,thoibinh,thoibinh,Ca Mau,Thoi Binh
10456,Cà Mau,Thành phố Cà Mau,Phường Tân Thành,Ward,camau,tanthanh,Ca Mau,Ca Mau
10457,Cà Mau,Thành phố Cà Mau,Xã Tân Thành,Commune,camau,tanthanh,Ca Mau,Ca Mau
176,Hà Nội,Huyện Gia Lâm,Thị trấn Yên Viên,Town,gialam,yenvien,Ha Noi,Gia Lam
178,Hà Nội,Huyện Gia Lâm,Xã Yên Viên,Commune,gialam,yenvien,Ha Noi,Gia Lam
10084,Kiên Giang,Huyện Vĩnh Thuận,Thị trấn Vĩnh Thuận,Town,vinhthuan,vinhthuan,Kien Giang,Vinh Thuan
10088,Kiên Giang,Huyện Vĩnh Thuận,Xã Vĩnh Thuận,Commune,vinhthuan,vinhthuan,Kien Giang,Vinh Thuan


Following Google Trend, "xã ..." is higher "thị trấn ..." significant. But "phường ..." is higher than "xã ..." and we have only one ward in the list.

In [88]:
# LIST_duplicated_ward_district_keys = df_duplicated_wards.ward_key.unique().tolist()

In [89]:
# DICT_duplicated_ward_keys = {}
# for ward_key in df_duplicated_wards['ward_key'].unique():
#     ward_levels = df_duplicated_wards[df_duplicated_wards.ward_key == ward_key]['ward_level_english'].tolist()
#     if 'Ward' in ward_levels:
#         default_ward_level = 'Ward'
#     elif 'Commune' in ward_levels:
#         default_ward_level = 'Commune'
#     else:
#         default_ward_level = 'Town'
#     DICT_duplicated_ward_keys[ward_key] = default_ward_level

In [98]:
DICT_duplicated_ward_keys = {}

for province_english in df_duplicated_wards.province_english.unique():
    district_data = {}
    tmp_df = df_duplicated_wards[df_duplicated_wards.province_english==province_english]
    for district_english in tmp_df.district_english.unique():
        ward_data = {}
        for ward_key in tmp_df[tmp_df.district_english==district_english]['ward_key'].unique():
            ward_levels = df_duplicated_wards[df_duplicated_wards.ward_key == ward_key]['ward_level_english'].tolist()
            if 'Ward' in ward_levels:
                default_ward_level = 'Ward'
            elif 'Commune' in ward_levels:
                default_ward_level = 'Commune'
            else:
                default_ward_level = 'Town'
            ward_data[ward_key] = default_ward_level
            
        district_data[district_english] = ward_data 
        
    DICT_duplicated_ward_keys[province_english] = district_data

In [100]:
DICT_duplicated_ward_keys['Ca Mau']

{'Thoi Binh': {'thoibinh': 'Commune'}, 'Ca Mau': {'tanthanh': 'Ward'}}

## Find province from district

- DICT_unique_long_district_alphanumerics `{'long_district_alphanumeric': 'province_key'}`
- DICT_unique_district_keys `{'district_key': 'province_key'}`
- DICT_not_unique_district_keys `{'district_key': {'province_key 1': ['ward key', ...], {'province_key 2': ['ward key', ...]}}`


Điều kiện 1: district_key not in ward_keys.
Điều kiện 2: district_count = 1, sau khi đã drop_duplicated [province_key, district_key]
Điều kiện 3: district_key not in province_keys của tỉnh khác.

Nếu district_key = province_key của chính nó:
>> Lấy danh sách long_district_alphanumerices (bao gồm Việt và English), nếu tồn tại trong address thì sau khi tìm được province_key sẽ không bị xóa province_key trong địa chỉ.


Nếu district_key != province_key của chính nó, tức là unique tuyệt đối:
>> Lấy danh sách unique_district_keys, nếu không tìm được province_key thì tìm unique_district_key để suy ngược ra province_key

In [101]:
print('province_keys:', len(province_keys))
print('district_keys:', len(district_keys))
print('ward_keys:', len(ward_keys))

province_keys: 63
district_keys: 669
ward_keys: 6648


### Use long_district_alphanumeric

Check if long_district_alphanumeric is in long_provinces_alphanumerics and long_ward_alphanumerics

In [102]:
df[~df['long_district_alphanumeric'].apply(lambda x: x in str(DICT_long_province_alphanumerics))].shape

(10547, 32)

In [103]:
df[~df['long_district_alphanumeric'].apply(lambda x: x in str(long_ward_alphanumerics))].shape

(10547, 32)

Yes, we will not mistake with long_district_alphanumeric with province or ward. But they can be duplicated.

In [104]:
df_long_district_alphanumeric_count = df[['province_key', 'long_district_alphanumeric']].drop_duplicates()['long_district_alphanumeric'].value_counts().reset_index()

In [105]:
df_long_district_alphanumeric_count

,long_district_alphanumeric,count
0,huyenchauthanh,10
1,huyenphongdien,2
2,huyentamduong,2
3,huyenanlao,2
4,huyenphuninh,2
...,...,...
680,thanhphocaobang,1
681,huyenhaquang,1
682,thanhphotanuyen,1
683,huyendautieng,1


In [106]:
unique_long_district_alphanumerics = df_long_district_alphanumeric_count[df_long_district_alphanumeric_count['count']==1]['long_district_alphanumeric'].values.tolist()
not_unique_long_district_alphanumerics = df_long_district_alphanumeric_count[df_long_district_alphanumeric_count['count']>1]['long_district_alphanumeric'].values.tolist()

In [107]:
df_unique_long_district_alphanumerics = df[df['long_district_alphanumeric'].isin(unique_long_district_alphanumerics)][['province_key', 'district_key', 'long_district_alphanumeric', 'long_district_alphanumeric_english']].drop_duplicates()
df_not_unique_long_district_alphanumerics = df[df['long_district_alphanumeric'].isin(not_unique_long_district_alphanumerics)][['province_key', 'district_key', 'long_district_alphanumeric', 'long_district_alphanumeric_english']].drop_duplicates()

In [108]:
df_unique_long_district_alphanumerics

,province_key,district_key,long_district_alphanumeric,long_district_alphanumeric_english
7229,ninhthuan,thuanbac,huyenthuanbac,thuanbacdistrict
7211,ninhthuan,ninhhai,huyenninhhai,ninhhaidistrict
7208,ninhthuan,ninhson,huyenninhson,ninhsondistrict
7200,ninhthuan,bacai,huyenbacai,bacaidistrict
7191,ninhthuan,phanrangthapcham,thanhphophanrangthapcham,phanrangthapchamcity
...,...,...,...,...
10530,camau,namcan,huyennamcan,namcandistrict
10465,camau,uminh,huyenuminh,uminhdistrict
10494,camau,tranvanthoi,huyentranvanthoi,tranvanthoidistrict
10462,camau,camau,thanhphocamau,camaucity


In [109]:
df_not_unique_long_district_alphanumerics

,province_key,district_key,long_district_alphanumeric,long_district_alphanumeric_english
9330,bentre,chauthanh,huyenchauthanh,chauthanhdistrict
9881,angiang,phutan,huyenphutan,phutandistrict
9951,angiang,chomoi,huyenchomoi,chomoidistrict
9936,angiang,chauthanh,huyenchauthanh,chauthanhdistrict
9120,longan,chauthanh,huyenchauthanh,chauthanhdistrict
223,hanoi,thanhtri,huyenthanhtri,thanhtridistrict
3132,vinhphuc,tamduong,huyentamduong,tamduongdistrict
9726,dongthap,tamnong,huyentamnong,tamnongdistrict
9804,dongthap,chauthanh,huyenchauthanh,chauthanhdistrict
792,caobang,baolam,huyenbaolam,baolamdistrict


In [110]:
df_unique_long_district_alphanumerics['district_key_equal_province_key'] = np.where(df_unique_long_district_alphanumerics['province_key']==df_unique_long_district_alphanumerics['district_key'], True, False)
df_not_unique_long_district_alphanumerics['district_key_equal_province_key'] = np.where(df_not_unique_long_district_alphanumerics['province_key']==df_not_unique_long_district_alphanumerics['district_key'], True, False)

In [111]:
df_unique_long_district_alphanumerics['district_key_equal_province_key'].value_counts()

district_key_equal_province_key
False    642
True      32
Name: count, dtype: int64

In [112]:
df_not_unique_long_district_alphanumerics['district_key_equal_province_key'].value_counts()

district_key_equal_province_key
False    30
Name: count, dtype: int64

In [113]:
DICT_unique_long_district_alphanumerics = {}
for row in df_unique_long_district_alphanumerics.itertuples():
    long_district_alphanumeric = row.long_district_alphanumeric
    alias_district_alphanumeric = re.sub(r'^thanhpho', 'tp.', long_district_alphanumeric, flags=re.IGNORECASE)
    alias_district_alphanumeric = re.sub(r'^quan', 'q.', alias_district_alphanumeric, flags=re.IGNORECASE)
    alias_district_alphanumeric = re.sub(r'^huyen', 'h.', alias_district_alphanumeric, flags=re.IGNORECASE)
    alias_district_alphanumeric = re.sub(r'^thixa', 'tx.', alias_district_alphanumeric, flags=re.IGNORECASE)
    
    DICT_unique_long_district_alphanumerics[long_district_alphanumeric] = row.province_key
    DICT_unique_long_district_alphanumerics[alias_district_alphanumeric] = row.province_key
    DICT_unique_long_district_alphanumerics[row.long_district_alphanumeric_english] = row.province_key

In [114]:
DICT_unique_long_district_alphanumerics

{'huyenthuanbac': 'ninhthuan',
 'h.thuanbac': 'ninhthuan',
 'thuanbacdistrict': 'ninhthuan',
 'huyenninhhai': 'ninhthuan',
 'h.ninhhai': 'ninhthuan',
 'ninhhaidistrict': 'ninhthuan',
 'huyenninhson': 'ninhthuan',
 'h.ninhson': 'ninhthuan',
 'ninhsondistrict': 'ninhthuan',
 'huyenbacai': 'ninhthuan',
 'h.bacai': 'ninhthuan',
 'bacaidistrict': 'ninhthuan',
 'thanhphophanrangthapcham': 'ninhthuan',
 'tp.phanrangthapcham': 'ninhthuan',
 'phanrangthapchamcity': 'ninhthuan',
 'huyenninhphuoc': 'ninhthuan',
 'h.ninhphuoc': 'ninhthuan',
 'ninhphuocdistrict': 'ninhthuan',
 'huyenthuannam': 'ninhthuan',
 'h.thuannam': 'ninhthuan',
 'thuannamdistrict': 'ninhthuan',
 'huyenbaoyen': 'laocai',
 'h.baoyen': 'laocai',
 'baoyendistrict': 'laocai',
 'thixasapa': 'laocai',
 'tx.sapa': 'laocai',
 'sapatown': 'laocai',
 'huyenbacha': 'laocai',
 'h.bacha': 'laocai',
 'bachadistrict': 'laocai',
 'huyenbaothang': 'laocai',
 'h.baothang': 'laocai',
 'baothangdistrict': 'laocai',
 'huyenbatxat': 'laocai',
 'h.b

In [115]:
df['long_district_alphanumeric_ward_key_unit'] = df['long_district_alphanumeric'] + '_' + df['ward_key']
long_district_alphanumeric_ward_key_unit_count = df['long_district_alphanumeric_ward_key_unit'].value_counts().reset_index()
unique_long_district_alphanumeric_ward_key_units = long_district_alphanumeric_ward_key_unit_count[long_district_alphanumeric_ward_key_unit_count['count']==1]['long_district_alphanumeric_ward_key_unit'].tolist()

In [116]:
long_district_alphanumeric_ward_key_unit_count

,long_district_alphanumeric_ward_key_unit,count
0,huyenchauthanh_chauthanh,4
1,huyenchauthanh_tanphu,3
2,huyenchauthanh_anhiep,3
3,huyenyenbinh_yenbinh,2
4,huyenngason_ngathanh,2
...,...,...
10478,huyenthapmuoi_truongxuan,1
10479,huyenthapmuoi_tankieu,1
10480,huyenthapmuoi_myhoa,1
10481,huyenthapmuoi_myquy,1


In [117]:
DICT_not_unique_long_district_alphanumerics = {}
for row in df_not_unique_long_district_alphanumerics.itertuples():
    long_district_alphanumeric = row.long_district_alphanumeric
    alias_district_alphanumeric = abbreviate_alphanumeric_prefix(long_district_alphanumeric)
    province_data = {}
    tmp_province_keys = df[df.long_district_alphanumeric==long_district_alphanumeric]['province_key'].unique()
    for province_key in tmp_province_keys:
        tmp_ward_keys = df[(df.long_district_alphanumeric==long_district_alphanumeric) & (df.province_key==province_key) & (df.long_district_alphanumeric_ward_key_unit.isin(unique_long_district_alphanumeric_ward_key_units))]['ward_key'].dropna().tolist()
        for ward_key in tmp_ward_keys:
            if re.search(r'^phuong\d{1,2}', ward_key):
                tmp_ward_keys.append(ward_key.replace('phuong', 'ward'))
                tmp_ward_keys.append(ward_key.replace('phuong', 'p.'))
        province_data[province_key] = tmp_ward_keys
    DICT_not_unique_long_district_alphanumerics[long_district_alphanumeric] = province_data
    DICT_not_unique_long_district_alphanumerics[alias_district_alphanumeric] = province_data
    DICT_not_unique_long_district_alphanumerics[row.long_district_alphanumeric_english] = province_data

In [118]:
DICT_not_unique_long_district_alphanumerics

{'huyenchauthanh': {'bentre': ['quoithanh',
   'phuocthanh',
   'tanthach',
   'quoison',
   'giaolong',
   'phutuc',
   'phuduc',
   'phuanhoa',
   'anphuoc',
   'thanhtrieu',
   'tuongda',
   'tienlong',
   'tamphuoc',
   'tienthuy',
   'huudinh',
   'sonhoa'],
  'angiang': ['vinhthanh',
   'vinhloi',
   'vinhnhuan',
   'hoabinhthanh',
   'binhhoa',
   'vinhbinh',
   'binhthanh',
   'vinhhanh',
   'candang',
   'vinhan',
   'anchau'],
  'longan': ['tamvu',
   'binhquoi',
   'hoaphu',
   'phungaitri',
   'vinhcong',
   'thuanmy',
   'hiepthanh',
   'phuoctanhung',
   'thanhphulong',
   'duongxuanhoi',
   'anluclong',
   'longtri',
   'thanhvinhdong'],
  'dongthap': ['caitauha',
   'annhon',
   'tannhuandong',
   'tanbinh',
   'tanphutrung',
   'phulong',
   'anphuthuan',
   'hoatan'],
  'tayninh': ['haoduoc',
   'longvinh',
   'ninhdien',
   'thanhlong',
   'thanhdien',
   'anbinh',
   'hoahoi',
   'tribinh',
   'hoathanh',
   'biengioi',
   'anco',
   'dongkhoi',
   'thaibinh',
   'p

In [119]:
LIST_contains_province_key_long_district_alphanumerics = []
for long_district_alphanumeric in df_unique_long_district_alphanumerics[df_unique_long_district_alphanumerics['district_key_equal_province_key']]['long_district_alphanumeric'].unique():
    alias_district_alphanumeric = abbreviate_alphanumeric_prefix(long_district_alphanumeric)
    LIST_contains_province_key_long_district_alphanumerics.append(long_district_alphanumeric)
    LIST_contains_province_key_long_district_alphanumerics.append(alias_district_alphanumeric)
LIST_contains_province_key_long_district_alphanumerics += df_not_unique_long_district_alphanumerics[df_not_unique_long_district_alphanumerics['district_key_equal_province_key']]['long_district_alphanumeric'].unique().tolist()

In [120]:
LIST_contains_province_key_long_district_alphanumerics

['thanhpholaocai',
 'tp.laocai',
 'thanhphobentre',
 'tp.bentre',
 'thanhphohagiang',
 'tp.hagiang',
 'thanhphosonla',
 'tp.sonla',
 'thanhphohungyen',
 'tp.hungyen',
 'thanhphothanhhoa',
 'tp.thanhhoa',
 'thanhphobaclieu',
 'tp.baclieu',
 'thanhphohoabinh',
 'tp.hoabinh',
 'huyendienbien',
 'h.dienbien',
 'thanhphoyenbai',
 'tp.yenbai',
 'thanhphocaobang',
 'tp.caobang',
 'thanhphobackan',
 'tp.backan',
 'thixaphutho',
 'tt.phutho',
 'thanhphotuyenquang',
 'tp.tuyenquang',
 'thanhphothaibinh',
 'tp.thaibinh',
 'thanhpholaichau',
 'tp.laichau',
 'thanhphothainguyen',
 'tp.thainguyen',
 'thanhpholangson',
 'tp.langson',
 'thanhphobacgiang',
 'tp.bacgiang',
 'thanhphobacninh',
 'tp.bacninh',
 'thanhphovinhlong',
 'tp.vinhlong',
 'thanhphotayninh',
 'tp.tayninh',
 'thanhphonamdinh',
 'tp.namdinh',
 'thanhphohaiduong',
 'tp.haiduong',
 'thanhphoninhbinh',
 'tp.ninhbinh',
 'thanhphohatinh',
 'tp.hatinh',
 'thixaquangtri',
 'tt.quangtri',
 'thanhphotravinh',
 'tp.travinh',
 'thanhphoquangnga

### Use district_key

In [121]:
df_district_filter = df[~df.district_key.apply(lambda x: x in str(long_ward_alphanumerics))].copy()

In [122]:
def check_district_equal_other_province_keys(province_key, district_key):
    tmp_province_keys = province_keys.copy()
    tmp_province_keys.remove(province_key)
    if district_key in tmp_province_keys:
        return True
    else:
        return False

In [123]:
df_district_filter['district_key_equal_other_province_keys'] = df_district_filter.apply(lambda x: check_district_equal_other_province_keys(x.province_key, x.district_key), axis=1)

In [124]:
df_district_filter['district_key_equal_other_province_keys'].value_counts()

district_key_equal_other_province_keys
False    2893
Name: count, dtype: int64

In [125]:
df_district_filter = df_district_filter[~df_district_filter['district_key_equal_other_province_keys']]

In [126]:
df_district_filter[df_district_filter.district_key=='quan5']

,province,district,ward,long_province,long_district,long_ward,short_district,short_ward,province_english,district_english,...,long_district_alphanumeric,long_ward_alphanumeric,long_province_alphanumeric_english,long_district_alphanumeric_english,long_ward_alphanumeric_english,province_alphanumeric,short_district_alphanumeric,short_ward_alphanumeric,long_district_alphanumeric_ward_key_unit,district_key_equal_other_province_keys
8818,Hồ Chí Minh,Quận 5,Phường 4,Thành phố Hồ Chí Minh,Quận 5,Phường 4,Quận 5,Phường 4,Ho Chi Minh,District 5,...,quan5,phuong4,hochiminhcity,district5,ward4,hochiminh,quan5,phuong4,quan5_phuong4,False
8819,Hồ Chí Minh,Quận 5,Phường 9,Thành phố Hồ Chí Minh,Quận 5,Phường 9,Quận 5,Phường 9,Ho Chi Minh,District 5,...,quan5,phuong9,hochiminhcity,district5,ward9,hochiminh,quan5,phuong9,quan5_phuong9,False
8820,Hồ Chí Minh,Quận 5,Phường 3,Thành phố Hồ Chí Minh,Quận 5,Phường 3,Quận 5,Phường 3,Ho Chi Minh,District 5,...,quan5,phuong3,hochiminhcity,district5,ward3,hochiminh,quan5,phuong3,quan5_phuong3,False
8821,Hồ Chí Minh,Quận 5,Phường 12,Thành phố Hồ Chí Minh,Quận 5,Phường 12,Quận 5,Phường 12,Ho Chi Minh,District 5,...,quan5,phuong12,hochiminhcity,district5,ward12,hochiminh,quan5,phuong12,quan5_phuong12,False
8822,Hồ Chí Minh,Quận 5,Phường 2,Thành phố Hồ Chí Minh,Quận 5,Phường 2,Quận 5,Phường 2,Ho Chi Minh,District 5,...,quan5,phuong2,hochiminhcity,district5,ward2,hochiminh,quan5,phuong2,quan5_phuong2,False
8823,Hồ Chí Minh,Quận 5,Phường 8,Thành phố Hồ Chí Minh,Quận 5,Phường 8,Quận 5,Phường 8,Ho Chi Minh,District 5,...,quan5,phuong8,hochiminhcity,district5,ward8,hochiminh,quan5,phuong8,quan5_phuong8,False
8824,Hồ Chí Minh,Quận 5,Phường 7,Thành phố Hồ Chí Minh,Quận 5,Phường 7,Quận 5,Phường 7,Ho Chi Minh,District 5,...,quan5,phuong7,hochiminhcity,district5,ward7,hochiminh,quan5,phuong7,quan5_phuong7,False
8825,Hồ Chí Minh,Quận 5,Phường 1,Thành phố Hồ Chí Minh,Quận 5,Phường 1,Quận 5,Phường 1,Ho Chi Minh,District 5,...,quan5,phuong1,hochiminhcity,district5,ward1,hochiminh,quan5,phuong1,quan5_phuong1,False
8826,Hồ Chí Minh,Quận 5,Phường 11,Thành phố Hồ Chí Minh,Quận 5,Phường 11,Quận 5,Phường 11,Ho Chi Minh,District 5,...,quan5,phuong11,hochiminhcity,district5,ward11,hochiminh,quan5,phuong11,quan5_phuong11,False
8827,Hồ Chí Minh,Quận 5,Phường 14,Thành phố Hồ Chí Minh,Quận 5,Phường 14,Quận 5,Phường 14,Ho Chi Minh,District 5,...,quan5,phuong14,hochiminhcity,district5,ward14,hochiminh,quan5,phuong14,quan5_phuong14,False


In [127]:
df_district_filter_count = df_district_filter[['province_key', 'district_key']].drop_duplicates()['district_key'].value_counts().reset_index()

In [128]:
df_district_filter_count

,district_key,count
0,hoangmai,2
1,tamnong,2
2,thuanbac,1
3,nguhanhson,1
4,hoavang,1
...,...,...
200,camranh,1
201,hamthuanbac,1
202,tuyphong,1
203,lagi,1


In [129]:
unique_district_keys = df_district_filter_count[df_district_filter_count['count']==1]['district_key'].values.tolist()
not_unique_district_keys = df_district_filter_count[df_district_filter_count['count']>1]['district_key'].values.tolist()

In [130]:
df_unique_district_keys = df[df['district_key'].isin(unique_district_keys)][['province_key', 'district_key']].drop_duplicates()
df_not_unique_district_keys = df[df['district_key'].isin(not_unique_district_keys)][['province_key', 'district_key']].drop_duplicates()

In [131]:
df_unique_district_keys

,province_key,district_key
7229,ninhthuan,thuanbac
7200,ninhthuan,bacai
7191,ninhthuan,phanrangthapcham
9305,bentre,bentre
9448,bentre,mocaybac
...,...,...
10127,cantho,omon
10274,soctrang,soctrang
10349,soctrang,nganam
10544,camau,ngochien


In [132]:
df_not_unique_district_keys

,province_key,district_key
101,hanoi,hoangmai
9726,dongthap,tamnong
3007,phutho,tamnong
5661,nghean,hoangmai


In [133]:
DICT_unique_district_keys = {}
for row in df_unique_district_keys.itertuples():
    district_key = row.district_key
    DICT_unique_district_keys[district_key] = row.province_key
    if re.search(r'quan\d{1,2}', district_key):
        DICT_unique_district_keys[district_key.replace('quan', 'q.')] = row.province_key
        DICT_unique_district_keys[district_key.replace('quan', 'district')] = row.province_key

In [134]:
DICT_unique_district_keys 

{'thuanbac': 'ninhthuan',
 'bacai': 'ninhthuan',
 'phanrangthapcham': 'ninhthuan',
 'bentre': 'bentre',
 'mocaybac': 'bentre',
 'mocaynam': 'bentre',
 'hoangsuphi': 'hagiang',
 'bacquang': 'hagiang',
 'bacme': 'hagiang',
 'lethuy': 'quangbinh',
 'tuyenhoa': 'quangbinh',
 'botrach': 'quangbinh',
 'quynhnhai': 'sonla',
 'tuyhoa': 'phuyen',
 'tuyan': 'phuyen',
 'myhao': 'hungyen',
 'chaudoc': 'angiang',
 'thoaison': 'angiang',
 'campha': 'quangninh',
 'mongcai': 'quangninh',
 'duchue': 'longan',
 'kientuong': 'longan',
 'mochoa': 'longan',
 'bathuoc': 'thanhhoa',
 'samson': 'thanhhoa',
 'bimson': 'thanhhoa',
 'quangxuong': 'thanhhoa',
 'thachthanh': 'thanhhoa',
 'nhuthanh': 'thanhhoa',
 'nhuxuan': 'thanhhoa',
 'baclieu': 'baclieu',
 'giarai': 'baclieu',
 'hongdan': 'baclieu',
 'lacthuy': 'hoabinh',
 'yenthuy': 'hoabinh',
 'bactuliem': 'hanoi',
 'thachthat': 'hanoi',
 'chuongmy': 'hanoi',
 'hoankiem': 'hanoi',
 'caugiay': 'hanoi',
 'namtuliem': 'hanoi',
 'nampo': 'dienbien',
 'muonglay': '

In [135]:
df['district_key_ward_key_unit'] = df['district_key'] + '_' + df['ward_key']
district_key_ward_key_unit_count = df['district_key_ward_key_unit'].value_counts().reset_index()
unique_district_key_ward_key_units = district_key_ward_key_unit_count[district_key_ward_key_unit_count['count']==1]['district_key_ward_key_unit'].tolist()

In [136]:
DICT_not_unique_district_keys = {}
for row in df_not_unique_district_keys.itertuples():
    district_key = row.district_key
    province_data = {}
    tmp_province_keys = df[df.district_key==district_key]['province_key'].unique()
    for province_key in tmp_province_keys:
        tmp_ward_keys = df[(df.district_key==district_key) & (df.province_key==province_key) & (df.district_key_ward_key_unit.isin(unique_district_key_ward_key_units))]['ward_key'].unique().tolist()
        for ward_key in tmp_ward_keys:
            if re.search(r'^phuong\d{1,2}', ward_key):
                tmp_ward_keys.append(ward_key.replace('phuong', 'p.'))
                tmp_ward_keys.append(ward_key.replace('phuong', 'ward'))
        province_data[province_key] = tmp_ward_keys
    
    DICT_not_unique_district_keys[district_key] = province_data
    if re.search(r'quan\d{1,2}', district_key):
        DICT_not_unique_district_keys[district_key.replace('quan', 'q.')] = province_data
        DICT_not_unique_district_keys[district_key.replace('quan', 'district')] = province_data

In [137]:
DICT_not_unique_district_keys

{'hoangmai': {'hanoi': ['thanhtri',
   'vinhhung',
   'dinhcong',
   'maidong',
   'tuongmai',
   'daikim',
   'tanmai',
   'hoangvanthu',
   'giapbat',
   'linhnam',
   'thinhliet',
   'tranphu',
   'hoangliet',
   'yenso'],
  'nghean': ['quynhlien',
   'quynhphuong',
   'quynhxuan',
   'quynhdi',
   'maihung',
   'quynhtrang',
   'quynhlap',
   'quynhloc',
   'quynhvinh',
   'quynhthien']},
 'tamnong': {'dongthap': ['phuhiep',
   'tancongsinh',
   'phuduc',
   'phuthanhb',
   'anhoa',
   'anlong',
   'phucuong',
   'phuninh',
   'phuthanha',
   'phutho',
   'hoabinh',
   'tramchim'],
  'phutho': ['hienquan',
   'hunghoa',
   'bacson',
   'thanhuyen',
   'lamson',
   'quanghuc',
   'huongnon',
   'tele',
   'thovan',
   'dinau',
   'danquyen',
   'vanxuan']}}

## Create alias province_keys

In [138]:
DICT_alias_province_keys = {'hcm':'hochiminh'}

## Save data


- DICT_long_province_alphanumerics `{'province alphanumeric': 'province key', ...}`
- DICT_long_district_alphanumerics `{'Province English': {'district alphanumeric': 'district key', ...}}`
- DICT_long_ward_alphanumerics `{'Province English': {'District English': {'long ward alphanumeric': 'ward key'}}}` 

- LIST_long_district_alphanumerics `['district alphanumeric', ...]`
- LIST_long_ward_alphanumerics `['long ward alphanumeric', ...]`

- LIST_province_keys_1 `['short province key', ...]`
- LIST_province_keys_2 `['short province key', ...]`
- LIST_province_keys_3 `['short province key', ...]`

- DICT_province_map `{'short province key': {'province': 'Hồ Chí Minh', 'long_province': 'Thành phố Hồ Chí Minh', ...}, ...}`
- DICT_district_map `{'Province English': {'district key': {'level english': {'district': 'Tân Bình', 'long_district': 'Quận Tân Bình', ...}}}}`
- DICT_ward_map `{'Province English':{'District English': {'ward key': {'level english': {'ward': 'Phú Trinh', 'long_ward': 'Phường Phú Trinh', ...}}}}}`

- LIST_duplicated_district_province_keys `['short province key', ...]`
- DICT_duplicated_district_keys

- LIST_duplicated_ward_district_keys `['short district key', ...]`
- DICT_duplicated_ward_keys

In [139]:
data = {
    'DICT_long_province_alphanumerics': DICT_long_province_alphanumerics,
    'DICT_long_district_alphanumerics': DICT_long_district_alphanumerics,
    'DICT_long_ward_alphanumerics': DICT_long_ward_alphanumerics,
    
    'LIST_safe_long_district_alphanumerics': LIST_safe_long_district_alphanumerics,
    'LIST_safe_long_ward_alphanumerics': LIST_safe_long_ward_alphanumerics,
    
    'LIST_province_keys_1': LIST_province_keys_1,
    'LIST_province_keys_2': LIST_province_keys_2,
    'LIST_province_keys_3': LIST_province_keys_3,
    
    'DICT_province_map': DICT_province_map,
    'DICT_district_map': DICT_district_map,
    'DICT_ward_map': DICT_ward_map,
    
    # 'LIST_duplicated_district_province_keys': LIST_duplicated_district_province_keys,
    'DICT_duplicated_district_keys': DICT_duplicated_district_keys,
    
    # 'LIST_duplicated_ward_district_keys': LIST_duplicated_ward_district_keys,
    'DICT_duplicated_ward_keys': DICT_duplicated_ward_keys,
    
    'DICT_unique_long_district_alphanumerics': DICT_unique_long_district_alphanumerics,
    'DICT_not_unique_long_district_alphanumerics': DICT_not_unique_long_district_alphanumerics,
    'LIST_contains_province_key_long_district_alphanumerics': LIST_contains_province_key_long_district_alphanumerics,
    'DICT_unique_district_keys': DICT_unique_district_keys,
    'DICT_not_unique_district_keys': DICT_not_unique_district_keys,
    
    'DICT_alias_province_keys': DICT_alias_province_keys
    
}

In [140]:
with open('../../vietadminunits/data/parse.pkl', 'wb') as f:
    pickle.dump(data, f)

In [141]:
DICT_duplicated_district_keys['Ha Tinh']

{'kyanh': {'default': 'Town',
  'levels': {'Town': ['kyha',
    'kyloi',
    'kyninh',
    'hungtri',
    'kytrinh',
    'kythinh',
    'kyphuong',
    'kyhoa',
    'kylong',
    'kylien',
    'kynam'],
   'District': ['kykhang',
    'kyvan',
    'kytrung',
    'kytho',
    'kytay',
    'kylac',
    'kyson',
    'lamhop',
    'kytan',
    'kythuong',
    'kyhai',
    'kythu',
    'kychau',
    'kydong',
    'kygiang',
    'kytien',
    'kyphu',
    'kybac',
    'kyxuan',
    'kyphong']}}}